<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Meta_Llama_3_8B_for_MEDAL_EVALUATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meta_Llama_3_8B_for_MEDAL-EVALUATOR.ipynb

In [ ]:
! pip install peft --quiet

!pip install -U flash-attn --no-build-isolation --quiet

#Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [ ]:
!pip install colab-env --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2"

peft_model_id = 'frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2'

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
#/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
#nrec= randint(0, len(eval_dataset))

nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

'antral follicle count </s><s>[INST] the present study was designed to evaluate the usefulness of immunoblotting and IEM techniques for the diagnosis of human cystic echinococcosis ce in patients from a nonendemic area of spain where the infection is not notifiable serum samples from three groups of patients were analyzed a positive control group was composed of sera from patients with surgically proven ce from a endemic area of spain the second group consisted of sera from patients with surgically proven ce obtained from a nonendemic area of spain and the third group was formed by sera'

In [ ]:
def similarity(ganswer,oanswer):

  qc0=str(ganswer).find('[INST]')

  ganswer=str(ganswer)[0:qc0-8]
  qc=str(ganswer).find('[/INST]')
  if qc>0:
    ganswer=ganswer[qc+8:len(ganswer)]

  print(f"Generated Answer:\n{ganswer}")

  if ganswer == oanswer:
    print("Match")
    match=1
  else:
    print("NO Match")
    match=0
  return match

In [ ]:
print(f"Query:\n{eval_dataset[nrec]['text']}")

oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")

ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

match=similarity(ganswer,oanswer)

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
else:
    ganswer=ganswer[qc+8:len(ganswer)]
ganswer

'antral follicle count </s><s>[INST] the present study was designed to evaluate the usefulness of immunoblotting and IEM techniques for the diagnosis of human cystic echinococcosis ce in patients from a nonendemic area of spain where the infection is not notifiable serum samples from three groups of patients were analyzed a positive control group was composed of sera from patients with surgically proven ce from a endemic area of spain the second group consisted of sera from patients with surgically proven ce obtained from a nonendemic area of spain and the third group was formed by sera'

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [ ]:
from tqdm import tqdm

#prompt =  eval_dataset[nrec]['text']


#outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
#                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
#                                  pad_token_id=tokenizer.eos_token_id)


def evaluate(sample):
    prompt =  sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer=outputs[0]['generated_text'][len(prompt):].strip()
    #print()
    #print(ganswer)
    print(f"Generated Answer original:\n{ganswer}")

    # molecule [/INST] phosphorylethanolamine </s><s>[INST]
    qc0=str(ganswer).find('[/INST]')
    qc1=str(ganswer).find('[INST]')

    ganswer=str(ganswer)[qc0+8:qc1-8]
    #print(f"Generated Answer corr:\n{ganswer}")

    oanswer=str(sample['label'])
    oanswer=oanswer[2:len(oanswer)-2]

    #print(f"Original Answer:\n{oanswer}")
    if ganswer == oanswer:
        #print()
        #print("Match")
        #print(f"Query:\n{sample['text']}")
        #print(f"Original Answer:\n{oanswer}")
        #print(f"Generated Answer:\n{ganswer}")
        return 1
    else:
        #print()
        #print("no Match")
        #print(f"Query:\n{sample['text']}")
        print(f"Original Answer:\n{oanswer}")
        print(f"Generated Answer:\n{ganswer}")
        return 0

#frankmorales2020/Meta-Llama-3-8B-MEDAL-flash-attention-2


#Original Answer:
#enteropathogenic
#Generated Answer:
#enteropathogenic e coli


success_rate = []
number_of_eval_samples = 100
# iterate over eval dataset and predict

#for n in tqdm(range(number_of_eval_samples)):
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

  1%|          | 1/100 [00:10<16:30, 10.01s/it]

Generated Answer original:
[/INST] function tests </s><s>[INST] we have developed a method for measuring the adhesion of platelets to purified collagen SF obtained from bovine tendon this method differs from others in that a platelet adhesion is measured in the absence of platelet aggregation and that the collagen substrate is in the form of flexible fibers rather than a rigid surface collagen fibrils were isolated from bovine tendon demineralized and solubilized in acetic acid they were then purified of impurities especially glycoprotein by passing a solution of the collagen through a column of sephadex g in acetic acid on which imp
Original Answer:
functional residual capacity
Generated Answer:
function tests


  2%|▏         | 2/100 [00:20<16:20, 10.01s/it]

Generated Answer original:
[/INST] pyogenic granuloma </s><s>[INST] the present study was undertaken to investigate the involvement of phospholipase a pla in the impairment of SM retention produced by a betaamyloid protein abeta microinfusion into the basal forebrain the effect of the pla inhibitor bromoenol lactone bel on the retention of PAH pah was examined pah was po twice the first time before abeta or bel microinfusion and the second time h after abeta microinfusion abeta microinfusion impaired pah retention which was prevented by pretreatment with bel in doses of and nmol bel also


  3%|▎         | 3/100 [00:30<16:11, 10.02s/it]

...

Generated Answer original:
[/INST] functional magnetic resonance </s><s>[INST] the pharmacokinetics of Pt pt from cisplatin ddp i.v. were studied in patients with PT using a new liquid chromatographyelectrochemical detection method the plasma samples were analyzed for pt by a CS and precise method which involved protein precipitation with trichloroacetic acid and ionpair highperformance liquid chromatography with a c column and uv PCD at nm along with electrochemical detection and a microcoulometer plug flow dialysis was used to establish a baseline range of pt C2 in plasma to which the measured pt C2 could be compared during
Original Answer:
echoplanar
Generated Answer:
functional magnetic resonance


 36%|███▌      | 36/100 [05:57<10:34,  9.92s/it]

Generated Answer original:
[/INST] carboxypeptidase a </s><s>[INST] thermostability of proteins is an important functional and ecological parameter for organisms living in thermophilic environments to investigate the molecular basis of thermal stability of proteins we have performed SDM of proteins from thermophilic and mesophilic bacteria using as a training data set protein pairs that are homologous to one another and differ in their optimal growth temperature we have focused on protein pairs for which crystal structures of both thermophilic and mesophilic homologues are known and for which there is experimental information on stability of the protein we have selected amino acid properties
Original Answer:
enzymesubstrate
Generated Answer:
carboxypeptidase a


 36%|███▌      | 36/100 [06:06<10:51, 10.18s/it]


KeyboardInterrupt: 

In [ ]:
#print(f"Accuracy: {accuracy*100:.2f}%")

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 100: 22.00%


When evaluated on 1000 samples from the evaluation dataset, our model achieved an impressive accuracy of TBD%. However, there's room for improvement.